In [1]:
!git clone https://github.com/AleSilva04/wv72_tf_201822717_201620127_201816502_20181d073_201815776.git

Cloning into 'wv72_tf_201822717_201620127_201816502_20181d073_201815776'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 163 (delta 73), reused 22 (delta 8), pack-reused 0
Receiving objects: 100% (163/163), 2.08 MiB | 5.58 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [8]:
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import LeerCsv as leer
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import Generar_Grafo as grafos
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import Dijkstra as dijks
from wv72_tf_201822717_201620127_201816502_20181d073_201815776 import Funciones_Agrupar as agrupar
import heapq as hq
import math

In [10]:
al = leer.leer_archivo("wv72_tf_201822717_201620127_201816502_20181d073_201815776/dataset/warehouses.csv")
pe = leer.leer_archivo("wv72_tf_201822717_201620127_201816502_20181d073_201815776/dataset/delivery-points.csv")

points = pe+al

print(points)

[(487, 555), (834, 940), (223, 206), (884, 259), (853, 197), (101, 292), (530, 423), (962, 524), (660, 111), (847, 108), (909, 103), (776, 109), (272, 93), (707, 293), (801, 490), (223, 960), (497, 12), (480, 769), (818, 797), (734, 123), (342, 668), (155, 553), (346, 765), (371, 599), (613, 806), (445, 0), (700, 948), (370, 451), (959, 770), (291, 626), (41, 431), (193, 227), (397, 612), (765, 422), (130, 846), (801, 710), (586, 5), (350, 660), (188, 301), (888, 404), (213, 715), (780, 590), (817, 831), (486, 155), (210, 264), (780, 499), (512, 900), (732, 782), (351, 460), (279, 310), (996, 230), (5, 318), (378, 838), (909, 765), (590, 826), (935, 926), (495, 61), (127, 589), (198, 233), (86, 827), (915, 734), (309, 252), (697, 195), (650, 550), (932, 129), (846, 309), (271, 288), (148, 554), (4, 752), (224, 157), (831, 871), (482, 229), (483, 753), (249, 455), (938, 417), (109, 881), (455, 640), (902, 476), (118, 57), (520, 481), (517, 276), (813, 9), (838, 735), (747, 710), (448, 8

In [14]:
#Generamos Grafo de un millon de nodos
G = grafos.generateGridGraph(1000)


#Agregamos los nodos de los almacenes
almacenes = []
for i in range(5000,5100):
  x,y = points[i]
  almacenes.append(y*1000+x)


#Agregamos los nodos de los puntos de entrega
ptsEntrega = []

for i in range(5000):
  x,y = points[i]
  ptsEntrega.append(y*1000 +x)

In [18]:
#Le asignamos un almacen a cada uno de los 5000 puntos de entrega
group =[-1]*5000
for i in range(5000):
  m=0
  d=agrupar.dist(points[i],points[5000])
  for j in range(1,100):
    d2 = agrupar.dist(points[i],points[5000+j])
    if d2 < d:
      d=d2
      m = j
  group[i]=m
  


In [19]:
#Agrupamos todos los nodos en su correspondiente grupo  
groupNodes = [[] for _ in range(100)]
for i in range(5000):
  g = group[i]
  groupNodes[g].append(ptsEntrega[i])

#Dividimos el Grafo General en subgrafos o regiones donde en cada una estan un almacen y sus respectivos puntos de entrega asignados
regions = [[]for _ in range(100)]
for i in range(100):
  groupNodes[i].append(almacenes[i])
  limits = agrupar.limites(groupNodes[i])
  for y in range(limits[0],limits[1]+1):
    for x in range(limits[2],limits[3]+1):
      regions[i].append(y*1000+x)

In [20]:
#Esta funcion halla el camino que se tiene que hacer para ir desde un nodo a otro desde una lista de adyacencia
def hallarCamino(u,v,G):
  camino=[]
  yu = int(u/1000)
  xu = u - yu*1000
  yv = int(v/1000)
  xv = v - yv*1000
  if yu<yv:
    if xu>xv:
      while xu>xv:
        camino.append(yu*1000+xu-1)
        xu-=1
      while yu< yv:
        camino.append((yu+1)*1000+xu)
        yu+=1
    else:
      while yu< yv:
        camino.append((yu+1)*1000+xu)
        yu+=1
      while xu<xv:
        camino.append(yu*1000+xu+1)
        xu+=1
  else:
    if xu>xv:
      while xu>xv:
        camino.append(yu*1000+xu-1)
        xu-=1
      while yu> yv:
        camino.append((yu-1)*1000+xu)
        yu-=1
    else:
      while yu> yv:
        camino.append((yu-1)*1000+xu)
        yu-=1
      while xu<xv:
        camino.append(yu*1000+xu+1)
        xu+=1

  return camino


In [21]:

#Esta funcion calcula el camino que se tiene que seguir de un nodo a otro dentro de un diccionario que contiene todos los caminos
def camino(s,u,dic):
  path=[]
  path.append(u)
  aux=dic[u]
  path.append(aux)
  while True:
    aux=dic[aux]
    if aux == s:
      break
    path.append(aux)
  return path

In [23]:

#Aca empieza el algortimo de vrp
vrp = []

#Vamos a calcular que deben de seguir los vehiculos para cada almacen, tomando solo los primeros 24 almacenes
for ind in range(24):
  
  #Primero aplicamos dijkstra desde almacen en el que estemos a todos los puntos de entrega de su region
  g = groupNodes[ind]
  path,_= dijks.dijkstra(regions[ind],g,G,almacenes[ind])
  caminos=[]
  paths=[]
  n=len(g)-1
  
  #De ahi calculamos los caminos de nodos que se tienen que seguir para ir desde el almacen hacia cada punto de entrega
  for i in range(n):
    a= camino(almacenes[ind],g[i],path)
    paths.append(a)

    
  #Despues eliminamos los caminos que tengan nodos que ya se estan vistando en otros caminos
  ptsVisitado = [False]*n
  idx=0
  for j,p in enumerate(paths):
    for i in range(n):
      if g[i] in p:
        ptsVisitado[i]=True
    idx=j
    if ptsVisitado.count(False)==0:
      break
  aux=n-1
  while aux>idx:
    paths.pop(len(paths)-1)
    aux-=1

    
  #Luego unimos caminos en pares sin repetir hasta unir todos, si tiene un numero de caminos impar el ultimo camino considera el regreso como el mismo camino de ida
  agregado = [False]*len(paths)
  for j,p in enumerate(paths):
    if agregado.count(False) ==1:
      c = paths[0].copy()
      c.reverse()
      c.pop(len(c)-1)
      cam = paths[0]+c
      caminos.append(cam)
      break
    if agregado.count(False) == 0:
      break
    if agregado[j] is True:
      continue
    d = 1000000
    y1 = int(p[0]/1000)
    x1 = p[0] - y1*1000
    m = 0
    #Calculamos el mejor par de caminos que se pueden ir dentro de los caminos que todavia no hemos agregado, consideramos mejor como que sus ultimos nodos son las cercanos
    #para asi a la hora de unirlos el recorrido agregado sea el menor posible
    for i in range(len(paths)):
      if p == paths[i]:
        continue
      if agregado[i] is True:
        continue
      node = paths[i][0]
      y2 = int(node/1000)
      x2 = node -y2*1000
      auxd = agrupar.dist((x1,y1),(x2,y2))
      if auxd < d:
        d = auxd
        m = i
        
    #Calculamos el camino extra que se debe de hacer para unir los caminos
    c = hallarCamino(p[0],paths[m][0],G)
    
    #unimos los 3 caminos, el par de caminos y el camino extra para la union
    cam = p.copy() + c.copy() + paths[m].copy()
    agregado[j]=True
    agregado[m]=True
    caminos.append(cam)
  #Se revisa ultima vez por si estan haciendo caminos innecsarios
  ultimaRev = [False]*n
  index=0
  for j,p in enumerate(caminos):
    for i in range(n):
      if g[i] in p:
        ultimaRev[i]=True
    index=j
    if ultimaRev.count(False)==0:
      break
  
  #Agregamos todos los caminos finales
  vrp.append(caminos)


  


In [24]:
#Finalmente imprimimos por cada almacen, la cantidad de vehiculos que se necesita, los caminos que debe de hacer y los kms por cada camino
for i,cams in enumerate(vrp):
  print(f"Almacen {i}:")
  print(f"Vehiculos: {len(cams)}")
  print("Caminos:")
  for p in cams:
    print(p)
    print(f"Kms del camino: {len(p)+2}")
  print("\n")

Almacen 0:
Vehiculos: 9
Caminos:
[40949, 39949, 38949, 37949, 36949, 35949, 34949, 34948, 34947, 34946, 34945, 34944, 34943, 34942, 34941, 34940, 34939, 34938, 34937, 34936, 34935, 34934, 34933, 34932, 34931, 34930, 34929, 34928, 34927, 34926, 34925, 34924, 34923, 34922, 34921, 34920, 34919, 34918, 34917, 34916, 34915, 40948, 40947, 40946, 39946, 39946, 38946, 37946, 36946, 35946, 34946, 34945, 34944, 34943, 34942, 34941, 34940, 34939, 34938, 34937, 34936, 34935, 34934, 34933, 34932, 34931, 34930, 34929, 34928, 34927, 34926, 34925, 34924, 34923, 34922, 34921, 34920, 34919, 34918, 34917, 34916, 34915]
Kms del camino: 84
[15896, 15897, 15898, 15899, 15900, 15901, 15902, 15903, 15904, 15905, 15906, 15907, 15908, 15909, 15910, 15911, 15912, 15913, 15914, 16914, 17914, 18914, 19914, 20914, 21914, 22914, 23914, 24914, 25914, 26914, 27914, 28914, 29914, 30914, 31914, 32914, 33914, 14896, 13896, 13897, 13898, 13899, 13900, 13901, 13902, 13903, 13904, 13905, 13905, 13906, 13907, 13908, 13909, 1